In [43]:
import pandas as pd
import numpy as np
import sys

sys.path.append('../src') 

data = pd.read_csv('..\\data\\all_politicians_aggregated_final.csv')

In [47]:
# remove all rows with NaN delivery_by_region
data = data[pd.notnull(data['delivery_by_region'])]

'''
There +100.000 ads without delivery_by_region because they have really low impressions
'''

'\nThere +100.000 ads without delivery_by_region because they have really low impressions\n'

In [50]:
geo = data.iloc[0].delivery_by_region
# turn str into lst
geo = eval(geo)
# add new key to point
impressions = float(data.iloc[0].impressions)
geo[0]['impressions'] = round(float(geo[0]["percentage"]) * impressions)

def calc_impressions_per_state(df):
    geo = df["delivery_by_region"]
    geo = eval(geo) #turn into obj
    impressions = df["impressions"]
    for region in geo:
        region["impressions"] = round(float(region["percentage"]) * impressions)
    return geo

In [54]:
from tqdm import tqdm 
tqdm.pandas()

# apply the transformation to all advertisements
data["geo_transf"] = data.progress_apply(calc_impressions_per_state, axis=1)

100%|██████████| 463403/463403 [01:52<00:00, 4121.30it/s] 


In [58]:
# now we need to aggregate the impressions per state for unique advertisement

#https://stackoverflow.com/questions/44348426/pandas-groupby-custom-function-to-each-series

test = data.sample(3, random_state=42)

test

,ad_creation_time,ad_creative_body,spend,impressions,delivery_by_region,demographic_distribution,page_id,page_name,bylines,id,spend_lo,spend_hi,impressions_lo,impressions_hi,geo_transf
456186,2020-09-24,“He shows up. He’s not just a politician. He’s...,1249.5,112499.5,"[{'percentage': '0.000185', 'region': 'Indiana...","[{'percentage': '0.000268', 'age': '18-24', 'g...",68563629337,Fred Upton,Upton for All of Us,638752283478441,1000,1499,100000,124999,"[{'percentage': '0.000185', 'region': 'Indiana..."
139401,2018-10-12,ALL DONATIONS MATCHED! A group of generous don...,149.5,3499.5,"[{'percentage': '0.000533', 'region': 'Alabama...","[{'percentage': '0.000533', 'age': '18-24', 'g...",1413682351987764,Andy Kim for Congress,Andy Kim for Congress,2178300565775654,100,199,3000,3999,"[{'percentage': '0.000533', 'region': 'Alabama..."
453938,2018-07-26,We have an important fundraising deadline comi...,49.5,499.5,"[{'percentage': '1', 'region': 'Wisconsin'}]","[{'percentage': '0.015873', 'age': '18-24', 'g...",7357041101,Tammy Baldwin,Tammy Baldwin for Senate,235255143963843,0,99,0,999,"[{'percentage': '1', 'region': 'Wisconsin', 'i..."


In [90]:
def aggregate_impressions_per_state(geographic_lists):
    res = {}
    geographic_data = np.asanyarray(geographic_lists)
    for geo in geographic_data:
        for region in geo:
            print(region)
            state = region["region"]
            if state in res:
                res[state] += region["impressions"]
            else:
                res[state] = region["impressions"]
    return res

grouped = data.groupby("ad_creative_body")["geo_transf"].agg(aggregate_impressions_per_state)
grouped = pd.DataFrame(grouped).reset_index()

KeyError: 'region'

In [88]:
grouped["geo_transf"][0]

{'Alabama': 2,
 'Oklahoma': 7,
 'Montana': 2,
 'Nevada': 4,
 'New Hampshire': 4,
 'New Jersey': 1562,
 'New Mexico': 9,
 'New York': 323,
 'North Carolina': 30,
 'Ohio': 13,
 'Oregon': 34,
 'Mississippi': 4,
 'Pennsylvania': 173,
 'Rhode Island': 7,
 'South Carolina': 17,
 'Tennessee': 2,
 'Texas': 21,
 'Utah': 9,
 'Vermont': 6,
 'Virginia': 67,
 'Washington': 54,
 'Missouri': 17,
 'Minnesota': 19,
 'Arizona': 17,
 'Hawaii': 15,
 'Arkansas': 4,
 'California': 349,
 'Colorado': 22,
 'Connecticut': 22,
 'Delaware': 11,
 'District of Columbia': 214,
 'Florida': 45,
 'Georgia': 19,
 'Idaho': 4,
 'Michigan': 19,
 'Illinois': 86,
 'Indiana': 6,
 'Iowa': 6,
 'Kansas': 4,
 'Kentucky': 9,
 'Louisiana': 7,
 'Maine': 7,
 'Maryland': 91,
 'Massachusetts': 142,
 'Wisconsin': 17}